In [ ]:
# read in merged fqs and parse file to extract TAM-target-site-EcoRI-reporter-flanking region
# then write these to a fasta file for alignment with reference

In [1]:
import sys

In [ ]:
# import all required modules

#!{sys.executable} -m pip install biopython
!{sys.executable} -m pip install editdistance

In [ ]:
# define sample fns for input and output

#t1_fns = {"/home/ec2-user/ngs_data/MiSeq_PE600_102323/1_cutmg_agx102/AGX_079_S79_L001.extendedFrags.fastq.gz": "/home/ec2-user/ngs_data/MiSeq_PE600_102323/2_parse_fasta/agx79.fasta", "/home/ec2-user/ngs_data/MiSeq_PE600_102323/1_cutmg_agx102/AGX_080_S80_L001.extendedFrags.fastq.gz": "/home/ec2-user/ngs_data/MiSeq_PE600_102323/2_parse_fasta/agx80.fasta"}
#t2_fns = {"/home/ec2-user/ngs_data/MiSeq_PE600_102323/1_cutmg_agx102/AGX_087_S87_L001.extendedFrags.fastq.gz": "/home/ec2-user/ngs_data/MiSeq_PE600_102323/2_parse_fasta/agx87.fasta", "/home/ec2-user/ngs_data/MiSeq_PE600_102323/1_cutmg_agx102/AGX_088_S88_L001.extendedFrags.fastq.gz": "/home/ec2-user/ngs_data/MiSeq_PE600_102323/2_parse_fasta/agx88.fasta"}
#t3_fns = {"/home/ec2-user/ngs_data/MiSeq_PE600_102323/1_cutmg_agx102/AGX_095_S95_L001.extendedFrags.fastq.gz": "/home/ec2-user/ngs_data/MiSeq_PE600_102323/2_parse_fasta/agx95.fasta", "/home/ec2-user/ngs_data/MiSeq_PE600_102323/1_cutmg_agx102/AGX_096_S96_L001.extendedFrags.fastq.gz": "/home/ec2-user/ngs_data/MiSeq_PE600_102323/2_parse_fasta/agx96.fasta"}
#t4_fns = {"/home/ec2-user/ngs_data/MiSeq_PE600_102323/1_cutmg_agx102/AGX_103_S103_L001.extendedFrags.fastq.gz": "/home/ec2-user/ngs_data/MiSeq_PE600_102323/2_parse_fasta/agx103.fasta", "/home/ec2-user/ngs_data/MiSeq_PE600_102323/1_cutmg_agx102/AGX_104_S104_L001.extendedFrags.fastq.gz": "/home/ec2-user/ngs_data/MiSeq_PE600_102323/2_parse_fasta/agx104.fasta"}
#t5_fns = {"/home/ec2-user/ngs_data/MiSeq_PE600_102323/1_cutmg_agx102/AGX_111_S111_L001.extendedFrags.fastq.gz": "/home/ec2-user/ngs_data/MiSeq_PE600_102323/2_parse_fasta/agx111.fasta", "/home/ec2-user/ngs_data/MiSeq_PE600_102323/1_cutmg_agx102/AGX_112_S112_L001.extendedFrags.fastq.gz": "/home/ec2-user/ngs_data/MiSeq_PE600_102323/2_parse_fasta/agx112.fasta"}



In [6]:
# Path to the reference FASTA file
ref_fasta = '/home/ec2-user/ngs_data/MiSeq_PE150_20240412/refs/2024Apr13_mismatch_lib.fa'
from collections import defaultdict
from Bio import SeqIO
import gzip
import re
from tqdm.notebook import tqdm
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import os

def read_ref_fasta(file_path):
    """Read a FASTA file and return a dictionary of sequences."""
    sequences = defaultdict(list)
    for record in SeqIO.parse(file_path, "fasta"):
        #print(record)
        sequences[record.id] = str(record.seq)
    #print(sequences)
    return sequences

read_1 = read_ref_fasta(ref_fasta)



In [7]:
print(read_1)

defaultdict(<class 'list'>, {'TCCACGTT': 'CAAAGTACgTCCACGTTCTTATGCAGGAGGGAGAGCTTGGCAGGGGGACCGGT', 'TCAGTGAG': 'CAAAGTACgTCAGTGAGCTTATGCTGGAGGGAGAGCTTGGCAGGGGGACCGGT', 'ACTCTCGC': 'CAAAGTACgACTCTCGCCTTATGCCGGAGGGAGAGCTTGGCAGGGGGACCGGT', 'AGACCCGG': 'CAAAGTACgAGACCCGGCTTATGCGAGAGGGAGAGCTTGGCAGGGGGACCGGT', 'TGATCTTC': 'CAAAGTACgTGATCTTCCTTATGCGTGAGGGAGAGCTTGGCAGGGGGACCGGT', 'CGGGTGTA': 'CAAAGTACgCGGGTGTACTTATGCGCGAGGGAGAGCTTGGCAGGGGGACCGGT', 'CGGGACTC': 'CAAAGTACgCGGGACTCCTTATGCGGAAGGGAGAGCTTGGCAGGGGGACCGGT', 'AAGCCATA': 'CAAAGTACgAAGCCATACTTATGCGGTAGGGAGAGCTTGGCAGGGGGACCGGT', 'CATACCCA': 'CAAAGTACgCATACCCACTTATGCGGCAGGGAGAGCTTGGCAGGGGGACCGGT', 'ATGGTTGT': 'CAAAGTACgATGGTTGTCTTATGCGGGTGGGAGAGCTTGGCAGGGGGACCGGT', 'TTCCACGT': 'CAAAGTACgTTCCACGTCTTATGCGGGGGGGAGAGCTTGGCAGGGGGACCGGT', 'CTAAGTGT': 'CAAAGTACgCTAAGTGTCTTATGCGGGACGGAGAGCTTGGCAGGGGGACCGGT', 'TGATTCTA': 'CAAAGTACgTGATTCTACTTATGCGGGAAGGAGAGCTTGGCAGGGGGACCGGT', 'CTGAATGT': 'CAAAGTACgCTGAATGTCTTATGCGGGATGGAGAGCTTGGCAGGGGGACCGGT', 'TTGG

In [26]:
# loop through provided in_dir and parse specified fq files and write a split fastq file 
# based on the error corrected mismatch barcode id

from Bio import SeqIO
import gzip
import re
from tqdm.notebook import tqdm
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import os
import Levenshtein
import editdistance

def fuzzy_match_id(query_id, ref_ids, max_mismatches=0):
    """Find a matching ID in ref_ids allowing for a certain number of mismatches."""
    for ref_id in ref_ids:
        if editdistance.eval(query_id, ref_id) <= max_mismatches:
            return ref_id
    return None

def parse_fq(in_dir, grep_str, out_dir):
    
    for filename in os.listdir(in_dir):
        
        if filename.endswith("extendedFrags.fastq.gz"): # only consider merged reads
        
            file_path = os.path.join(in_dir, filename)
    
            with gzip.open(file_path, "rt") as handle:

                print(file_path)
                
                record_lst = []
                record_dict = defaultdict(list)
                
                for record in tqdm(SeqIO.parse(handle, "fastq")):

                    seq = str(record.seq)
                    match = re.search(grep_str, seq)

                    if match != None:

                        #print(match)
                        tar_region = re.findall(grep_str, seq)[0]
                        mm_bc = tar_region[9:17]
                        
                        ref_id = fuzzy_match_id(mm_bc, read_1.keys())
                        #print(mm_bc)
                        #record_id = record.id
                        #print(ref_id)
                        if ref_id != None:
                            record_dict[ref_id].append(record)
                        
                        #if ref_id != None:
                            
                        #    out_fp = out_dir + "/" + filename.split(".")[0] + "_" + ref_id + ".fastq"
                        #    with open(out_fp, "a") as output_handle:
                        #        SeqIO.write(record, output_handle, "fastq")
                        
                for ref_id, record_lst in record_dict.items():
                
                    out_fp = out_dir + "/" + filename.split(".")[0] + "_" + ref_id + ".fastq"
                    
                    with open(out_fp, "w") as output_handle:
                        
                        SeqIO.write(record_lst, output_handle, "fastq")
                    
                
                        
parse_1 = parse_fq("/home/ec2-user/ngs_data/MiSeq_PE150_20240412/1_cutmg/merged_fastq", "CAAAGTACG[ATCG]{8}[ATCG]+ACCGGT",
                   "/home/ec2-user/ngs_data/MiSeq_PE150_20240412/4_split_fq")



/home/ec2-user/ngs_data/MiSeq_PE150_20240412/1_cutmg/merged_fastq/HEK_210.extendedFrags.fastq.gz


0it [00:00, ?it/s]

/home/ec2-user/ngs_data/MiSeq_PE150_20240412/1_cutmg/merged_fastq/HEK_211.extendedFrags.fastq.gz


0it [00:00, ?it/s]

/home/ec2-user/ngs_data/MiSeq_PE150_20240412/1_cutmg/merged_fastq/A549_210_extendedFrags.fastq.gz


0it [00:00, ?it/s]

/home/ec2-user/ngs_data/MiSeq_PE150_20240412/1_cutmg/merged_fastq/A549_211_extendedFrags.fastq.gz


0it [00:00, ?it/s]

In [ ]:
# loop through provided in_dir and parse specified fq files and output single file with fasta where header specifies
# the mismatch id

from Bio import SeqIO
import gzip
import re
from tqdm.notebook import tqdm
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import os
import Levenshtein
import editdistance

def fuzzy_match_id(query_id, ref_ids, max_mismatches=2):
    """Find a matching ID in ref_ids allowing for a certain number of mismatches."""
    for ref_id in ref_ids:
        if editdistance.eval(query_id, ref_id) <= max_mismatches:
            return ref_id
    return None

def parse_fq(in_dir, grep_str, out_dir):
    
    for filename in os.listdir(in_dir):
        
        if filename.endswith("extendedFrags.fastq.gz"): # only consider merged reads
        
            file_path = os.path.join(in_dir, filename)
    
            with gzip.open(file_path, "rt") as handle:

                print(file_path)
                
                record_lst = []

                for record in tqdm(SeqIO.parse(handle, "fastq")):

                    seq = str(record.seq)
                    match = re.search(grep_str, seq)

                    if match != None:

                        #print(match)
                        tar_region = re.findall(grep_str, seq)[0]
                        mm_bc = tar_region[9:17]
                        ref_id = fuzzy_match_id(mm_bc, read_1.keys())
                        #print(mm_bc)
                        #record_id = record.id
                        #print(ref_id)
                        
                        if ref_id != None:
                            
                            new_record = SeqRecord(
                                Seq(tar_region),
                                id = ref_id)

                            #print(record)
                            
                            record_lst.append(record)
                
            with open(out_fp, "w") as output_handle:
                SeqIO.write(record_lst, output_handle, "fasta")


parse_1 = parse_fq("/home/ec2-user/ngs_data/MiSeq_PE150_20240412/1_cutmg/test", "CAAAGTACG[ATCG]{8}[ATCG]+ACCGGT",
                   "/home/ec2-user/ngs_data/MiSeq_PE150_20240412/2_split_fa")



In [28]:
# now read in the parsed query fasta and the reference fasta
# based on the mismatch-bc, compute the LD and use as a proxy for editing efficiency

from Bio import SeqIO
import Levenshtein
from tqdm.notebook import tqdm
import os
from Bio import SeqIO
import Levenshtein
import editdistance
from collections import defaultdict
import re

out_dict = {}

def read_fasta(file_path):
    """Read a FASTA file and return a dictionary of sequences."""
    sequences = defaultdict(list)
    for record in SeqIO.parse(file_path, "fasta"):
        #print(record)
        sequences[record.id].append(str(record.seq))
    #print(sequences)
    return sequences

def fuzzy_match_id(query_id, ref_ids, max_mismatches=2):
    """Find a matching ID in ref_ids allowing for a certain number of mismatches."""
    for ref_id in ref_ids:
        if editdistance.eval(query_id, ref_id) <= max_mismatches:
            return ref_id
    return None

def calculate_levenshtein(ref_sequences, query_sequences):
    """Calculate Levenshtein distance between reference and query sequences."""
    grep_str = r"CTTATGC[ATCG]+ACCGGT"
    results = defaultdict(list)
    for query_id, query_seq_lst in tqdm(query_sequences.items()):
        ref_id = fuzzy_match_id(query_id, ref_sequences.keys())
        if ref_id:
            
            #print(len(query_seq_lst)) # number of reads that are assigned to a mismatch ref id
            
            for query_seq in query_seq_lst:

                    ref_seq = ref_sequences[ref_id][0]
                    #print(ref_seq)
                    tar_ref_seq = re.findall(grep_str, ref_seq)[0][7:-6]
                    #tar_ref_seq = ref_seq[0][25:-6]
                    #print(tar_ref_seq)
                    tar_query_seq = query_seq[24:-6]
                    #print(tar_query_seq)
                    if len(tar_ref_seq) != len(tar_query_seq): # only consider reads with indels
                        
                        ld = Levenshtein.distance(tar_ref_seq.upper(), tar_query_seq.upper())
                        results[ref_id].append(ld)
                        #print(ref_id)
                        #print(tar_ref_seq)
                        #print(tar_query_seq)
                        #print(ld)
                        
                    else:
                        
                        ld = 0
                        results[ref_id].append(ld)

        #else:
        #    results[query_id] = (None, None)  # No corresponding reference sequence found
    
    #print(results)
    
    out_dict[filename] = results
    #return results

# Path to the reference FASTA file
ref_fasta = '/home/ec2-user/ngs_data/MiSeq_PE150_20240412/refs/2024Apr13_mismatch_lib.fa'
# Path to the directory containing query FASTA files
query_directory = '/home/ec2-user/ngs_data/MiSeq_PE150_20240412/2_parse'

# Read the reference sequences
ref_sequences = read_fasta(ref_fasta)
#print(ref_sequences)

# Loop through all files in the directory containing query FASTA files
for filename in tqdm(os.listdir(query_directory)):
    if filename.endswith("L001"):  # Ensure processing only FASTA files
        print(filename)
        file_path = os.path.join(query_directory, filename)
        query_sequences = read_fasta(file_path)
        #print(query_sequences)
        levenshtein_distances = calculate_levenshtein(ref_sequences, query_sequences)
        #print(f"Levenshtein distances for {filename}:")
        #for seq_id, ld in levenshtein_distances.items():
        #    if ld is not None:
        #        print(f"  {seq_id}: {ld}")
        #    else:
        #        print(f"  {seq_id}: Reference sequence not found")
        

print(out_dict)


  0%|          | 0/32 [00:00<?, ?it/s]

AGX_10_S10_L001


  0%|          | 0/106 [00:00<?, ?it/s]

AGX_21_S21_L001


  0%|          | 0/538 [00:00<?, ?it/s]

...

AGX_07_S7_L001


  0%|          | 0/11197 [00:00<?, ?it/s]

AGX_01_S1_L001


  0%|          | 0/14034 [00:00<?, ?it/s]

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [29]:
import pandas as pd

ee_dict = {} # dict of series

for sample, ld_dict in out_dict.items():
    
    #print(sample)
    sample_sries = pd.Series()
    for mm_bc, ld_lst in ld_dict.items():
        
        #print(ld_lst[0:100])
        #print(mm_bc)
        edit_eff_proxy = 1 - (ld_lst.count(0) / len(ld_lst))
        #print(edit_eff_proxy)
        sample_sries[mm_bc] = edit_eff_proxy
        ee_dict[sample] = sample_sries
        
ee_df = pd.DataFrame(ee_dict)
print(ee_df)

ee_df.to_csv("/home/ec2-user/ngs_data/MiSeq_PE150_20240412/3_outs/2024Apr13_ld_defined_ee.csv")
        
        
        

/tmp/ipykernel_11378/3680186298.py:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  sample_sries = pd.Series()
/tmp/ipykernel_11378/3680186298.py:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  sample_sries = pd.Series()
/tmp/ipykernel_11378/3680186298.py:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  sample_sries = pd.Series()
/tmp/ipykernel_11378/3680186298.py:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  sample_sries = pd.Series()
/tmp/ipykernel_11378/3680186298.py:8: FutureWarning: The default dtype for empty Series 

          AGX_10_S10_L001  AGX_21_S21_L001  AGX_28_S28_L001  AGX_04_S4_L001  \
AAACCCAA         0.250000         0.375000         0.378336        0.243478   
AACCACAT              NaN         0.400000         0.200000        0.146341   
AAGCCATA         0.200000         0.347150         0.382304        0.377065   
AATTCCTC         0.285714         0.448529         0.431515        0.355699   
ACAGCCGC         0.142857         0.319444         0.278539        0.226138   
...                   ...              ...              ...             ...   
TTCCACGT              NaN         0.375000         0.173913        0.073529   
TTCTCTCC         0.250000         0.317647         0.496718        0.266169   
TTGGGATC         0.300000         0.205607         0.274322        0.448240   
TTTAGTCC         0.125000         0.222222         0.270588        0.323599   
TTTGGTTA         0.375000         0.384615         0.337778        0.223469   

          AGX_31_S31_L001  AGX_24_S24_L001  AGX_20_

/tmp/ipykernel_11378/3680186298.py:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  sample_sries = pd.Series()


In [ ]:
# read in .csv with annotations of each alignment and its corresponding tile

def read_align_annot(in_fn):
    
    in_df = pd.read_csv(in_fn, index_col = 0, 
                   header = None)
    in_dict = in_df.to_dict().get(1)
    print(in_dict)
    
    return in_dict
    
read_metadata = read_align_annot("/home/ec2-user/ngs_data/MiSeq_PE500_2024Apr01/4_variant_count/translate_sample_sheet.csv")



In [ ]:
# now you want to convert DNA --> amino acid sequence , but make sure to trim to keep frame in tact

from Bio.Seq import Seq

def translate_seq(in_dict, meta_dict):
    
    aa_dict = defaultdict(list)
    
    for sample, list_dna in tqdm(in_dict.items()):
        
        tile_lookup = meta_dict.get(sample)
        
        print(tile_lookup)
        
        if tile_lookup == "t1": # you should change the process script to only parse in-frame seqs prior to alignment
            
            #print(sample)
            for seq in tqdm(list_dna):
                
                dna_seq = Seq(seq[0:252])
                #print(dna_seq)
                aa_seq = dna_seq.translate()
                #print(aa_seq)
                aa_dict[sample].append(aa_seq)
                 
        elif tile_lookup == "t2": # you should change the process script to only parse in-frame seqs prior to alignment
            
            for seq in tqdm(list_dna):
                
                dna_seq = Seq(seq[0:249])
                #print(dna_seq)
                aa_seq = dna_seq.translate()
                #print(aa_seq)
                aa_dict[sample].append(aa_seq)
                
        elif tile_lookup == "t3": # you should change the process script to only parse in-frame seqs prior to alignment
            
            for seq in tqdm(list_dna):
                
                dna_seq = Seq(seq[2:261])
                #print(dna_seq)
                aa_seq = dna_seq.translate()
                #print(aa_seq)
                aa_dict[sample].append(aa_seq)
                
        elif tile_lookup == "t4": # you should change the process script to only parse in-frame seqs prior to alignment
            
            for seq in tqdm(list_dna):
                
                dna_seq = Seq(seq[0:237])
                #print(dna_seq)
                aa_seq = dna_seq.translate()
                #print(aa_seq)
                aa_dict[sample].append(aa_seq)
                                
        elif tile_lookup == "t5": # you should change the process script to only parse in-frame seqs prior to alignment
            
            for seq in tqdm(list_dna):
                
                dna_seq = Seq(seq[0:276])
                print(dna_seq)
                aa_seq = dna_seq.translate()
                print(aa_seq)
                aa_dict[sample].append(aa_seq)  
                
    return aa_dict
            
translate_1 = translate_seq(apr2024_read, read_metadata)
        
        

In [ ]:
# now with the list of aa_seqs in each tile; compile a dataframe with samples as columns and freq as entries

def count_aa_freq(in_dict):
    
    master_dict = defaultdict(dict)
    
    for sample, aa_seq_lst in in_dict.items():
        
        #print(sample)
        #print(aa_seq_lst[0:100])
        
        freq_dict = defaultdict(int)
        
        for aa_seq in aa_seq_lst:
            
            #print(str(aa_seq))
            
            if str(aa_seq) not in freq_dict.keys():
                
                #print(aa_seq)
                
                freq_dict[str(aa_seq)] = 1
                
            elif str(aa_seq) in freq_dict.keys():
                
                #print(aa_seq)
                freq_dict[str(aa_seq)] += 1
    
        master_dict[sample].update(freq_dict)
    
    master_df = pd.DataFrame.from_dict(master_dict)
    
    print(master_df)
    
    return master_df
                   
freq_1 = count_aa_freq(translate_1)


In [ ]:
from matplotlib import pyplot as plt

plt.hist(freq_1["AGX-01_S1_L001"], bins = 100)
plt.yscale("log")


In [ ]:
freq_1.to_csv("/home/ec2-user/ngs_data/MiSeq_PE500_2024Apr01/4_variant_count/2024Apr05_tile_variant_abundance.csv")

In [ ]:
# read in frequency dataframe to analyze variant abundance across amino acid position

import pandas as pd

def read_df(in_fp):
    
    in_df = pd.read_csv(in_fp, index_col = 0)
    print(in_df)
    
    return in_df
    
read_1 = read_df("../../ngs_data/MiSeq_PE500_011224/4_variant_count/2024Jan16_tile_variant_abundance_mut_call.csv")


In [ ]:
#!{sys.executable} -m pip install levenshtein

from Levenshtein import distance as ld


In [ ]:

def add_mut_col(in_df):
        
    tile_seq_dict = {"t1": "MIRNKAFVVRLYPNAAQTELINRTLGSARFVYNHFLARRIAAYKESGKGLTYGQTSSELTLLKQAEETSWLSEVDKFALQNSLK", 
                     "t2": "KNLETAYKNFFRTVKQSGKKVGFPRFRKKRTGESYRTQFTNNNIQIGEGRLKLPKLGWVKTKGQQDIQGKILNVTVRRIHEGH", 
                     "t3": "YEASVLCEVEIPYLPAAPKFAAGVDVGIKDFAIVTDGVRFKHEQNPKYYRSTLKRLRKAQQTLSRRKKGSARYGKAKTKLARIHKR", 
                     "t4": "RIVNKRQDFLHKLTTSLVREYEIIGTEHLKPDNMRKNRRLALSISDAGWGEFIRQLEYKAAWYGRLVSKVSPYFPSSQL",
                     "t5": "LCHDCGFKNPEVKNLAVRTWTCPNCGETHDRDENAALNIRREALVAAGISDTLNAHGGYVRPASAGNGLRSENHATLVVGGSGPPKKKRKV*"}
    
    
    variant_seqs = list(in_df.index)
    
    #print(variant_seqs[0:100])
    
    for variant_seq in tqdm(variant_seqs):
        
        dist_dict = {}
        
        for tile_id, tile_seq in tile_seq_dict.items():
            
            dist = ld(variant_seq, tile_seq)
            #print(dist)
            dist_dict[tile_seq] = dist
            
        #print(dist_dict)
        
        match = min(dist_dict, key=dist_dict.get)
        #print(match)
            
        pos = 1
        
        #print(seq_str)
        
        mast_str = ""
        
        for aa_1 in variant_seq:
            
            if aa_1 != match[pos-1]:
            
                mut_str = match[pos-1] + str(pos) + aa_1
                #print(mut_str)
                mast_str += mut_str
            
            pos += 1
        
        print(mast_str)
        
        in_df.loc[variant_seq, "mut_call"] = mast_str

        
    print(in_df["mut_call"])
    
    return in_df

add_1 = add_mut_col(freq_1)



In [ ]:
# you should now normalize each column by the column sum (proportion of variants in each group)
# then you can use this metric to compare between the gfp-pos vs. gfp-neg groups and normalize to the wt

print(add_1)

In [ ]:
add_1.to_csv("/home/ec2-user/ngs_data/MiSeq_PE500_2024Apr01/4_variant_count/2024Apr04_tile_variant_abundance_mut_call.csv")

In [ ]:
# let's normalize the dataframe prior to filtering variants below a certain frequency threshold


def norm_df(in_df):
    
    samples = list(in_df.drop("mut_call", axis = 1))
    print(in_df)
    
    norm_df = pd.DataFrame(index = in_df.index)
    
    for sample in samples:
        
        sries_sum = in_df[sample].sum()
        #print(sries_sum)
        
        sries_norm = in_df[sample] / sries_sum
        
        #print(sries_norm)
        norm_df[sample] = sries_norm
    
    #print(norm_df)
    norm_df["mut_call"] = in_df["mut_call"]
    
    return norm_df
        
norm_1 = norm_df(freq_1)



In [ ]:
print(norm_1)

norm_1.to_csv("/home/ec2-user/ngs_data/MiSeq_PE500_2024Apr01/4_variant_count/2024Apr04_tile_variant_abundance_mut_call_norm.csv")



In [ ]:
import numpy as np
from matplotlib import pyplot as plt

for sample in list(norm_1):
    
    plt.hist(norm_1[sample], bins = 100)
    plt.xlim(0, .1)

    
    
    

In [ ]:
# now filter dfs by frequency cutoffs

def filter_df(in_df, cutoff):
    
    #print(in_df)
    mut_call = in_df["mut_call"]
    filter_df = pd.DataFrame(index = in_df.index)
    
    samples = list(in_df)
    
    for sample in samples:
        
        if sample != "mut_call":
            
            sries = in_df[sample]
            sries_filter = sries[sries >= cutoff]
            filter_df[sample] = sries_filter
            
        
    print(len(filter_df))
    drop_df = filter_df.dropna(how = "all")
    print(len(drop_df))
    
    drop_df["mut_call"] = mut_call
    
    return drop_df
    
filter_1 = filter_df(norm_1, 0.001)
filter_2 = filter_df(norm_1, 0.01)
filter_3 = filter_df(norm_1, 0.05)
    
    
    

In [ ]:
print(filter_1)

In [ ]:
# write filtered dfs to file

filter_1.to_csv("/home/ec2-user/ngs_data/MiSeq_PE500_2024Apr01/4_variant_count/2024Apr04_tile_variant_abundance_mut_call_filter_1e-3.csv")
filter_2.to_csv("/home/ec2-user/ngs_data/MiSeq_PE500_2024Apr01/4_variant_count/2024Apr04_tile_variant_abundance_mut_call_filter_1e-2.csv")
filter_3.to_csv("/home/ec2-user/ngs_data/MiSeq_PE500_2024Apr01/4_variant_count/2024Apr04_tile_variant_abundance_mut_call_filter_5e-2.csv")



In [ ]:
# parse amino acid sequence and compute frequency at each position (so freq_df with index as amino acid identity)
# column number is length of tile and entry is the frequency of variants found at each position

from tqdm.notebook import tqdm

def freq_df_loc(in_df, samples, tile_len):

    sub_df = in_df[samples].dropna(how = "any")
    print(len(sub_df))
    amino_acids = ["A", "R", "N", "D", "C", "Q", "E", "G", "H", "I", "L", "K", "M", "F", "P", "S",
                "T", "W", "Y", "V", "*"]
    
    freq_df = pd.DataFrame(index = amino_acids, columns = range(0, tile_len)).fillna(0)
    #print(freq_df)
    
    for index, row in tqdm(sub_df.iterrows()):
        
        #print(index)
        avg = (row[samples[0]] + row[samples[1]]) / 2
        #print(avg)
        pos = 0
        for base in index:
            
            #print(base)
            freq_df.loc[base, pos] += avg
            pos += 1
        
    #print(freq_df)
    return freq_df
        
freq_t1 = freq_df_loc(freq_1, ["AGX_73_S1_L001_t1", "AGX_78_S6_L001_t1"], 84)
#freq_t2 = freq_df_loc(read_1, ["agx87", "agx88"], 83)
#freq_t3 = freq_df_loc(read_1, ["agx95", "agx96"], 86)
#freq_t4 = freq_df_loc(read_1, ["agx103", "agx104"], 79)
#freq_t5 = freq_df_loc(read_1, ["agx111", "agx112"], 92)




In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np

def plt_hmap(in_df):
    
    norm_df = in_df / in_df.sum()
    #print(norm_df)
    #print(freq_t1)
    log_df = np.log10(in_df)
    #print(log_df)
    fig, ax = plt.subplots(figsize=(20,20))  
    #sns.heatmap(norm_df, xticklabels = True, cmap = "crest")
    #plt.show()
    sns.heatmap(log_df, xticklabels = True, cmap = "crest", vmin = 3, vmax = 14)
    
    return norm_df

hmap_t1 = plt_hmap(freq_t1)
#hmap_t2 = plt_hmap(freq_t2)
#hmap_t3 = plt_hmap(freq_t3)
#hmap_t4 = plt_hmap(freq_t4)
#hmap_t5 = plt_hmap(freq_t5)


In [ ]:
# also plot the normalized frequency of "positively charged residues across the tile"

def plt_pos_charge(in_df):
    
    t_df = in_df.T
    #print(t_df)
    
    sub_df = t_df[["K", "R", "H"]]
    sub_df['Total'] = sub_df.sum(axis=1)
    print(sub_df)
    
    plt.scatter(sub_df.index, sub_df["Total"])
    plt.scatter(sub_df.index, sub_df["K"])
    plt.scatter(sub_df.index, sub_df["R"])
    plt.scatter(sub_df.index, sub_df["H"])
    
plt_t1 = plt_pos_charge(hmap_t1)
#plt_t2 = plt_pos_charge(hmap_t2)
#plt_t3 = plt_pos_charge(hmap_t3)
#plt_t4 = plt_pos_charge(hmap_t4)
#plt_t5 = plt_pos_charge(hmap_t5)

In [ ]:
# now plot the diversity across the positions -- this should show mutated regions

from scipy.stats import entropy

def plt_mut_diversity(in_df, title):
    
    t_df = in_df.T
    for pos, row in t_df.iterrows():
        
        H = entropy(row, base=2)
        t_df.loc[pos, "entrpy"] = H
        
    #print(t_df)
    
    plt.scatter(t_df.index, t_df["entrpy"])
    plt.ylim(-0.1, 2)
    #plt.savefig("../../ngs_data/MiSeq_PE600_102323/4_outs/" + title, bbox_inches = "tight")
    plt.ylabel("variant diversity (bits)")
    plt.xlabel("amino acid position")
    plt.title(title.split(".")[0])
    plt.show()
        
H_t1 = plt_mut_diversity(freq_t1, "tile1_diversity_plot.pdf")
#H_t2 = plt_mut_diversity(freq_t2, "tile2_diversity_plot.pdf")
#H_t3 = plt_mut_diversity(freq_t3, "tile3_diversity_plot.pdf")
#H_t4 = plt_mut_diversity(freq_t4, "tile4_diversity_plot.pdf")
#H_t5 = plt_mut_diversity(freq_t5, "tile5_diversity_plot.pdf")


